# Semantic Kernel மற்றும் OpenBnB MCP Server இணைப்பு

இந்த நோட்புக் Semantic Kernel-ஐ OpenBnB MCP சர்வரை பயன்படுத்தி உண்மையான Airbnb தங்குமிடங்களை MCPStdioPlugin மூலம் தேடுவதற்கான முறையை விளக்குகிறது. LLM அணுகலுக்காக, இது Azure AI Foundry-ஐ பயன்படுத்துகிறது. உங்கள் சூழல் மாறிகளை அமைக்க, [Setup Lesson](/00-course-setup/README.md) ஐ பின்பற்றலாம்.


## தேவையான தொகுப்புகளை இறக்குமதி செய்யுங்கள்


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP பிளகின் இணைப்பை உருவாக்குதல்

நாம் [OpenBnB MCP சர்வர்](https://github.com/openbnb-org/mcp-server-airbnb) ஐ MCPStdioPlugin மூலம் இணைக்கப் போகிறோம். இந்த சர்வர் @openbnb/mcp-server-airbnb தொகுப்பின் மூலம் Airbnb தேடல் செயல்பாட்டை வழங்குகிறது.


## கிளையன்டை உருவாக்குதல்

இந்த மாதிரியில், நாங்கள் எங்கள் LLM அணுகலுக்காக Azure AI Foundry ஐ பயன்படுத்துவோம். உங்கள் சூழல் மாறிகள் சரியாக அமைக்கப்பட்டுள்ளன என்பதை உறுதிப்படுத்துங்கள்.


## சூழல் அமைப்பு

Azure OpenAI அமைப்புகளை கட்டமைக்கவும். கீழே உள்ள சூழல் மாறிகள் அமைக்கப்பட்டுள்ளன என்பதை உறுதிப்படுத்தவும்:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP ஒருங்கிணைப்பை புரிந்துகொள்வது

இந்த நோட்புக் **உண்மையான OpenBnB MCP சர்வருடன்** இணைக்கிறது, இது உண்மையான Airbnb தேடல் செயல்பாட்டை வழங்குகிறது.

### இது எப்படி செயல்படுகிறது:

1. **MCPStdioPlugin**: MCP சர்வருடன் தரவுகளை அனுப்ப/பெற தரவோட்டத்தைப் பயன்படுத்துகிறது
2. **உண்மையான NPM தொகுப்பு**: `@openbnb/mcp-server-airbnb`-ஐ npx மூலம் பதிவிறக்கம் செய்து இயக்குகிறது
3. **நிகழ்நிலை தரவுகள்**: Airbnb API-களிலிருந்து உண்மையான சொத்து தரவுகளை திருப்புகிறது
4. **செயல்பாட்டு கண்டுபிடிப்பு**: MCP சர்வரில் கிடைக்கும் செயல்பாடுகளை ஏஜென்ட் தானாகவே கண்டறிகிறது

### கிடைக்கக்கூடிய செயல்பாடுகள்:

OpenBnB MCP சர்வர் பொதுவாக வழங்குகிறது:
- **search_listings** - இடம் மற்றும் அளவுகோளின் அடிப்படையில் Airbnb சொத்துகளைத் தேடுதல்
- **get_listing_details** - குறிப்பிட்ட சொத்துகளின் விரிவான தகவல்களைப் பெறுதல்
- **check_availability** - குறிப்பிட்ட தேதிகளுக்கு கிடைப்பதைச் சரிபார்த்தல்
- **get_reviews** - சொத்துகளுக்கான மதிப்புரைகளைப் பெறுதல்
- **get_host_info** - சொத்து உரிமையாளர்களின் தகவல்களைப் பெறுதல்

### முன்பதிவுகள்:

- உங்கள் கணினியில் **Node.js** நிறுவப்பட்டிருக்க வேண்டும்
- MCP சர்வர் தொகுப்பை பதிவிறக்க **இணைய இணைப்பு** தேவை
- **NPX** கிடைக்க வேண்டும் (இது Node.js உடன் வருகிறது)

### இணைப்பைச் சோதித்தல்:

MCP சர்வரை கையேடு முறையில் சோதிக்க, நீங்கள் இயக்கலாம்:
```bash
npx -y @openbnb/mcp-server-airbnb
```

இது OpenBnB MCP சர்வரை பதிவிறக்கி தொடங்கும், பின்னர் Semantic Kernel உண்மையான Airbnb தரவுகளுக்காக அதனுடன் இணைக்கிறது.


## OpenBnB MCP சர்வருடன் ஏஜென்டை இயக்குதல்

இப்போது, OpenBnB MCP சர்வருடன் இணைக்கும் AI ஏஜென்டை இயக்குவோம், இது ஸ்டாக்ஹோமில் 2 பெரியவர்கள் மற்றும் 1 குழந்தைக்கு உண்மையான Airbnb தங்குமிடங்களைத் தேடுகிறது. தேடல் அளவுகோள்களை மாற்ற `user_inputs` பட்டியலை மாற்றுவதற்கு தயங்க வேண்டாம்.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# சுருக்கம்
வாழ்த்துக்கள்! நீங்கள் Model Context Protocol (MCP) பயன்படுத்தி உண்மையான தங்குமிடம் தேடலுடன் ஒருங்கிணைக்கும் AI முகவரியை வெற்றிகரமாக உருவாக்கியுள்ளீர்கள்:

## பயன்படுத்தப்பட்ட தொழில்நுட்பங்கள்:
- Semantic Kernel - Azure OpenAI உடன் புத்திசாலி முகவரிகளை உருவாக்க
- Azure AI Foundry - LLM திறன்கள் மற்றும் உரையாடல் முடிவுகளுக்காக
- MCP (Model Context Protocol) - கருவி ஒருங்கிணைப்புக்கான தரநிலையான நெறிமுறை
- OpenBnB MCP Server - உண்மையான Airbnb தேடல் செயல்பாட்டுக்காக
- Node.js/NPX - வெளிப்புற MCP சர்வரை இயக்க

## நீங்கள் கற்றுக்கொண்டது:
- MCP ஒருங்கிணைப்பு: Semantic Kernel முகவரிகளை வெளிப்புற MCP சர்வர்களுடன் இணைத்தல்
- நேரடி தரவுகளின் அணுகல்: APIகள் மூலம் உண்மையான Airbnb சொத்துகளை தேடுதல்
- நெறிமுறை தொடர்பு: முகவர் மற்றும் MCP சர்வருக்கு இடையிலான stdio தொடர்பு பயன்படுத்துதல்
- செயல்பாட்டு கண்டுபிடிப்பு: MCP சர்வர்களிலிருந்து கிடைக்கும் செயல்பாடுகளை தானாகவே கண்டறிதல்
- ஸ்ட்ரீமிங் பதில்கள்: நேரடியாக செயல்பாட்டு அழைப்புகளை பதிவு செய்தல் மற்றும் பதிவு செய்தல்
- HTML ரெண்டரிங்: முகவர் பதில்களை ஸ்டைலான அட்டவணைகள் மற்றும் தொடர்புடைய காட்சிகளுடன் வடிவமைத்தல்

## அடுத்த படிகள்:
- கூடுதல் MCP சர்வர்களை (வானிலை, விமானங்கள், உணவகங்கள்) ஒருங்கிணைத்தல்
- MCP மற்றும் A2A நெறிமுறைகளை இணைக்கும் பல முகவர் அமைப்பை உருவாக்குதல்
- உங்கள் சொந்த தரவூட்டங்களுக்கு தனிப்பயன் MCP சர்வர்களை உருவாக்குதல்
- அமர்வுகளுக்கு இடையில் நிலையான உரையாடல் நினைவுகளை செயல்படுத்துதல்
- MCP சர்வர் ஒருங்கிணைப்புடன் Azure Functions-க்கு முகவரியை வெளியிடுதல்
- பயனர் அங்கீகாரம் மற்றும் முன்பதிவு திறன்களை சேர்த்தல்



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளுங்கள். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
